## 1. 聚合方法
- 分组后得到的GroupBy对象可以使用聚合方法进行数据聚合，以下是一些常用的聚合方法：
![title](img/数据聚合.png)
- GroupBy对象还可以调用被分组对象里自带的任何方法（非聚合方法也可以）
- 自定义的聚合函数，只需使用`aggregate()`或`agg()`方法传入自定义函数，分组结果的各个切片调用自定义函数并返回运算结果
- 上表中的聚合方法，其实就是使用`agg()`方法调用聚合方法，被调用的聚合方法名称以字符串形式传入`agg()`，如`agg('mean')`
- 返回结果禁用索引：聚合运算后得到的结果默认都是以分组键作为索引，可在分组时使用`as_index=False`禁止将分组键作为索引而只作为普通的列，相当于对结果`reset_index()`

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({
    'key1': list('ABBCBCAA'),
    'key2': list('YZXYZXYZ'),
    'data1': np.random.randint(100, size=8),
    'data2': np.random.randint(10, size=8)
})
df

,key1,key2,data1,data2
0,A,Y,18,0
1,B,Z,76,5
2,B,X,68,8
3,C,Y,23,5
4,B,Z,34,6
5,C,X,98,5
6,A,Y,81,8
7,A,Z,7,4


In [3]:
# 自定义函数进行聚合操作
def test(arr):
    if len(arr) > 1:
        return max(arr) - min(arr)
    else:
        return arr


df.groupby(['key1', 'key2']).agg(test)

data1  data2
key1 key2              
A    Y        63      8
     Z         7      4
B    X        68      8
     Z        42      1
C    X        98      5
     Y        23      5

In [4]:
# 以下语句相当于df.groupby(['key1','key2']).mean()
df.groupby(['key1', 'key2']).agg('mean')

data1  data2
key1 key2              
A    Y      49.5    4.0
     Z       7.0    4.0
B    X      68.0    8.0
     Z      55.0    5.5
C    X      98.0    5.0
     Y      23.0    5.0

In [5]:
# 分组时可以禁止将分组键作为索引
df.groupby(['key1', 'key2'],as_index=False).agg('mean')

,key1,key2,data1,data2
0,A,Y,49.5,4.0
1,A,Z,7.0,4.0
2,B,X,68.0,8.0
3,B,Z,55.0,5.5
4,C,X,98.0,5.0
5,C,Y,23.0,5.0


## 2.  应用多个聚合方法
如果需要对数据中不同的列分别应用不同的聚合方法，可以使用`agg()`来实现
- **对所有列应用多种聚合方法：**  
将多种方法的方法名以数组形式传入`agg()`，返回结果中的列将会以方法名来命名
- 若需要自定义返回的列名，可以传入一组元组列表，元组元素分别为自定义名称和方法名
- **对每个列应用不同聚合方法：**  
以字典形式传入列名和方法名，其中列名作为key，方法名作为value，多个方法的话由方法名组成列表，若要返回自定义列名则使用元组，`{'列名':[('自定义列名','方法名'),'方法名']}`
- 只有在列上应用了多个方法时，才会返回层次化数据

In [6]:
# 同时应用多种聚合方法，注意已有的聚合方法名以字符串形式传入，列名默认为方法名
df.groupby(['key1', 'key2']).agg(['mean', 'count', test])

data1            data2           
           mean count test  mean count test
key1 key2                                  
A    Y     49.5     2   63   4.0     2    8
     Z      7.0     1    7   4.0     1    4
B    X     68.0     1   68   8.0     1    8
     Z     55.0     2   42   5.5     2    1
C    X     98.0     1   98   5.0     1    5
     Y     23.0     1   23   5.0     1    5

In [7]:
# 应用多种聚合方法，并自定义列名
df.groupby(['key1', 'key2']).agg([('平均数', 'mean'), ('计数', 'count')])

data1    data2   
            平均数 计数   平均数 计数
key1 key2                  
A    Y     49.5  2   4.0  2
     Z      7.0  1   4.0  1
B    X     68.0  1   8.0  1
     Z     55.0  2   5.5  2
C    X     98.0  1   5.0  1
     Y     23.0  1   5.0  1

In [8]:
# 对每个列应用不同聚合方法
df.groupby(['key1', 'key2']).agg({
    'data1': [('最小', 'min'), ('计数', 'count')],
    'data2': ['mean', test]
})

data1    data2     
             最小 计数  mean test
key1 key2                    
A    Y       18  2   4.0    8
     Z        7  1   4.0    4
B    X       68  1   8.0    8
     Z       34  2   5.5    1
C    X       98  1   5.0    5
     Y       23  1   5.0    5

## 3. apply()方法
`apply()` 将对象拆分成多个片段，然后对各片段分别应用传入的方法，最后将得到的结果再组合到一起。  
**`apply()`和`agg()`区别：**  
- `apply()` 是将从表格中拆分出来的一部分数据（DataFrame或者Series）应用到传入的方法上
- `agg()` 是将表格的一列数据应用到传入的方法上

In [14]:
df

,key1,key2,data1,data2
0,A,Y,18,0
1,B,Z,76,5
2,B,X,68,8
3,C,Y,23,5
4,B,Z,34,6
5,C,X,98,5
6,A,Y,81,8
7,A,Z,7,4


In [78]:
def func_sort(df,columns='data1'):
    return df.sort_values(columns)
df.groupby(['key1','key2']).apply(func_sort,columns=['data1','data2'])

key1 key2  data1  data2
key1 key2                          
A    Y    0    A    Y     18      0
          6    A    Y     81      8
     Z    7    A    Z      7      4
B    X    2    B    X     68      8
     Z    1    B    Z     76      5
          4    B    Z     34      6
C    X    5    C    X     98      5
     Y    3    C    Y     23      5

In [81]:
for x,y in df.groupby(['key1','key2']):
    print(func_sort(y,columns=['data1','data2']))

  key1 key2  data1  data2
0    A    Y     18      0
6    A    Y     81      8
  key1 key2  data1  data2
7    A    Z      7      4
  key1 key2  data1  data2
2    B    X     68      8
  key1 key2  data1  data2
4    B    Z     34      6
1    B    Z     76      5
  key1 key2  data1  data2
5    C    X     98      5
  key1 key2  data1  data2
3    C    Y     23      5
